### Создание слоев, функций активации и функции потерь для нейронной сети

В настоящее время существует достаточно ограниченное количество слоев, функций активации и функций потерь для нейронных сетей. Необходимо разрабатывать свои функции для того, чтобы сделать более или менее серьезный проект.

Пример, который я привожу будет решен с помощью PyTorch. Мы будем делать линейный слой.

Официальный туториал находиться на сайте [PyTorch](https://pytorch.org/docs/stable/notes/extending.html)

Этот ноутбук доступен в моем Github репозитории:
```
git clone https://github.com/andreiliphd/reinforcement-content.git
```

Если нет Git, то его нужно установить.

Linux:
```
sudo apt-get update
sudo apt-get install git
```

Windows: скачайте Git с сайта [git-scm.com](https://git-scm.com/download/win).

Если вы нашли ошибку на сайте, ее можно исправить самостоятельно сделав Pull Request в Git.

### Этапы решения задачи

1. Определяем класс с собственным алгоритмом дифференцирования.
2. Переопределяем `forward`.
3. Переопределяем `backward`.
4. Определяем модуль.
5. Переопределяем `__init__`.
6. Переопределяем `forward`.
7. Тестирование.


In [15]:
import torch
from torch import nn

### Определяем класс с собственным алгоритмом дифференцирования.

In [9]:
# Inherit from Function
class LinearFunction(torch.autograd.Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

### Переопределяем forward

Пишем формулу прямого распространения.

### Переопределяем backward
Пишем формулу обратного распространения. Обратите внимание, что мы берем как полный дериватив, так и частные деривативы по отношению к параметрам оптимизации.


### Определяем модуль

Модуль - это компонента, которую мы будем вставлять в нашу нейронную сеть.

In [10]:
class Linear(torch.nn.Module):
    def __init__(self, input_features, output_features, bias=True):
        super(Linear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features

        # nn.Parameter is a special kind of Tensor, that will get
        # automatically registered as Module's parameter once it's assigned
        # as an attribute. Parameters and buffers need to be registered, or
        # they won't appear in .parameters() (doesn't apply to buffers), and
        # won't be converted when e.g. .cuda() is called. You can use
        # .register_buffer() to register buffers.
        # nn.Parameters require gradients by default.
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            # You should always register all possible parameters, but the
            # optional ones can be None if you want.
            self.register_parameter('bias', None)

        # Not a very smart way to initialize weights
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)

    def forward(self, input):
        # See the autograd section for explanation of what happens here.
        return LinearFunction.apply(input, self.weight, self.bias)

    def extra_repr(self):
        # (Optional)Set the extra information about this module. You can test
        # it by printing an object of this class.
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )


### Переопределяем __init__

В `__init__` объявляем параметры оптимизации, так называемые `torch.nn.Parameter`.

### Переопределяем forward в модуле
Вызываем нашу функцию `autograd` `LinearFunction.apply(input, self.weight, self.bias)`.

### Тестирование

In [16]:
tensor = torch.randn([3,64])

In [12]:
fc = Linear(64,100)

In [13]:
fc(tensor)

tensor([[-1.5266e-01, -2.2930e-01,  1.3016e-01,  1.9424e-01, -8.2268e-01,
          2.9127e-01, -8.3403e-01,  7.9582e-01,  1.3244e+00,  4.4504e-01,
          6.9587e-01,  1.0573e-01, -1.7380e-02,  1.8936e-01, -6.0043e-01,
         -2.2504e-01,  2.6544e-01,  4.8095e-02, -3.6776e-01,  7.6825e-01,
          4.0249e-01, -2.3658e-01, -3.0179e-01, -8.2391e-01, -6.0442e-01,
          5.4096e-01, -4.8182e-01,  8.1980e-01, -5.0751e-01,  5.3642e-02,
         -9.1196e-02, -3.5150e-01,  1.3570e-01,  7.3673e-01,  2.3271e-01,
          4.2033e-01,  8.1786e-01, -8.5281e-01,  4.9003e-02, -4.7420e-01,
         -2.5353e-01,  3.8539e-01, -8.0665e-01,  5.8173e-01,  1.0536e-01,
          2.1014e-02, -4.8601e-02,  4.7198e-02, -1.9768e-01, -2.5104e-01,
         -6.4469e-02, -1.1550e+00,  3.9481e-01,  8.5703e-01, -4.7345e-01,
         -5.5124e-01,  1.0197e+00, -9.6282e-01,  4.3211e-01, -3.2549e-01,
          2.2929e-01,  2.9515e-01,  7.4135e-01, -3.9694e-01, -3.9768e-02,
          3.7301e-01,  1.0603e-01, -9.